# Import


In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import cufflinks

cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

# Read Data

In [2]:
develop = pd.read_csv("CORBL_2000-01-01_2021-12-31.csv", engine='python')
develop.head()

,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
0,1,2021-12-29,71,4798.0,3958278.7,851.4,815.0,851.4
1,2,2021-12-28,56,2596.0,2163466.2,856.0,821.3,849.9
2,3,2021-12-27,82,8257.0,6898740.4,877.0,825.0,856.0
3,4,2021-12-26,58,3820.0,3258775.5,875.0,842.8,860.0
4,5,2021-12-23,103,9501.0,8142395.8,890.3,849.7,851.0


# Sort Data in Ascending Order

In [3]:
develop = develop.sort_values(by="Date")
develop.head()

,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
861,862,2011-08-17,1,10.0,3820.0,382.0,382.0,382.0
860,861,2011-11-09,1,10.0,3750.0,375.0,375.0,375.0
859,860,2011-11-13,3,70.0,25130.0,368.0,354.0,354.0
858,859,2011-11-14,2,40.0,13700.0,347.0,341.0,341.0
857,858,2011-11-15,2,40.0,13220.0,335.0,329.0,329.0


# Handle Missing NAs

In [4]:
develop = develop.dropna()
develop.tail()

,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
4,5,2021-12-23,103,9501.0,8142395.8,890.3,849.7,851.0
3,4,2021-12-26,58,3820.0,3258775.5,875.0,842.8,860.0
2,3,2021-12-27,82,8257.0,6898740.4,877.0,825.0,856.0
1,2,2021-12-28,56,2596.0,2163466.2,856.0,821.3,849.9
0,1,2021-12-29,71,4798.0,3958278.7,851.4,815.0,851.4


# Plot Trend

In [5]:
develop.iplot(kind="line",x="Date",y="Close Price",xTitle="Date", yTitle="Close Price", title="Development Bank Trend")

# Next price

In [6]:
develop['Next Price'] = develop.shift(-1)['Close Price']

In [7]:
develop = develop.dropna()

# Rename Columns

In [8]:
develop.columns

Index(['S.N.', 'Date', 'Total Transactions', 'Total Traded Shares',
       'Total Traded Amount', 'Max. Price', 'Min. Price', 'Close Price',
       'Next Price'],
      dtype='object')

In [9]:
develop.rename(columns = {'S.N.':'SN','Total Transactions':'TTrans', 'Total Traded Shares':'TTS',
                              'Total Traded Amount':'TTA','Close Price':'ClosePrice','Next Price':'NextPrice',
                     'Max. Price':'MaxPrice','Min. Price':'MinPrice'}, inplace = True)

# Prepared Feature

In [10]:
features = ['ClosePrice',"Date"]
outFeature = ['NextPrice',"Date"]

In [11]:
# take close price as feature and next price as out feature
X, Y = develop[features], develop[outFeature] 
X.set_index("Date", inplace=True)
Y.set_index("Date", inplace=True)

# Normalize Data

In [12]:
#Normalize data using standard scalar.
ss = StandardScaler()

X["ClosePrice"] = ss.fit_transform(X)
Y["NextPrice"] = ss.fit_transform(Y)


 # Train Test Split

Using train_test_split of sklearn will split data randomly so the sequence is lost.

In [19]:
def train_test_split(x,y,train_size):
    n = int(len(x)*train_size)
    trainx = x[:n]
    testx = x[n:]
    trainy=y[:n]
    testy=y[n:]
    return trainx,testx,trainy,testy

In [20]:
#Split train, validation, test set from X,Y to 80%, 10%, 10%

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.9)

In [21]:
X_train.shape,X_test.shape,X.shape

((774, 1), (87, 1), (861, 1))

# Convert to Matrix

In [22]:
# convert into dataset matrix
def convertToMatrix(data, step):
    data = data.to_numpy()
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

step=5

trainX,trainY =convertToMatrix(X_train,step)
testX,testY =convertToMatrix(X_test,step)
# validX,validY = convertToMatrix(X_valid,step)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# validX = np.reshape(validX, (validX.shape[0], 1, validX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

trainX.shape,testX.shape #,validX.shape

((769, 1, 5), (82, 1, 5))

# Modeling

In [23]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN


# SimpleRNN model
model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(1,step), activation="relu"))
model.add(Dense(8, activation="relu")) 
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 32)                1216      
                                                                 
 dense_2 (Dense)             (None, 8)                 264       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1,489
Trainable params: 1,489
Non-trainable params: 0
_________________________________________________________________


In [24]:
# we have 90% of data as train and if we take 11.11% from train, that will be equal to 10% of overall data as validation
model.fit(trainX,trainY, validation_split=0.111,#validation_data=(validX,validY),
          epochs=100, batch_size=16, verbose=2)
trainPredict = model.predict(trainX)
testPredict= model.predict(testX)
predicted=np.concatenate((trainPredict,testPredict),axis=0)

Epoch 1/100
43/43 - 1s - loss: 0.0573 - val_loss: 0.0209 - 1s/epoch - 31ms/step
Epoch 2/100
43/43 - 0s - loss: 0.0030 - val_loss: 0.0115 - 83ms/epoch - 2ms/step
Epoch 3/100
43/43 - 0s - loss: 0.0025 - val_loss: 0.0110 - 88ms/epoch - 2ms/step
Epoch 4/100
43/43 - 0s - loss: 0.0021 - val_loss: 0.0148 - 83ms/epoch - 2ms/step
Epoch 5/100
43/43 - 0s - loss: 0.0019 - val_loss: 0.0116 - 83ms/epoch - 2ms/step
Epoch 6/100
43/43 - 0s - loss: 0.0018 - val_loss: 0.0212 - 86ms/epoch - 2ms/step
Epoch 7/100
43/43 - 0s - loss: 0.0017 - val_loss: 0.0090 - 85ms/epoch - 2ms/step
Epoch 8/100
43/43 - 0s - loss: 0.0016 - val_loss: 0.0145 - 83ms/epoch - 2ms/step
Epoch 9/100
43/43 - 0s - loss: 0.0015 - val_loss: 0.0135 - 85ms/epoch - 2ms/step
Epoch 10/100
43/43 - 0s - loss: 0.0015 - val_loss: 0.0125 - 86ms/epoch - 2ms/step
Epoch 11/100
43/43 - 0s - loss: 0.0015 - val_loss: 0.0083 - 90ms/epoch - 2ms/step
Epoch 12/100
43/43 - 0s - loss: 0.0014 - val_loss: 0.0077 - 107ms/epoch - 2ms/step
Epoch 13/100
43/43 - 0s -

3/3 [==============================] - 0s 2ms/step


# Evaluate

Smaller the better.

In [25]:
test_score = model.evaluate(testX, testPredict, verbose=0)
print(test_score)

1.750361398249016e-14


# RMSE of Test

In [26]:
from sklearn.metrics import mean_squared_error

mean_squared_error(ss.inverse_transform(testX[:,:,0].flatten()), 
                   ss.inverse_transform(testPredict.flatten()), squared=False)

73.0615535395047

# Predict Trend

In [27]:
test=pd.DataFrame()
test["pred"]=ss.inverse_transform(testPredict.flatten())
test["real"]=ss.inverse_transform(testX[:,:,0])
test["date"]=X_test.index[:-5]
test.iplot(kind="line", x="date")

# Train Trend

In [28]:
train=pd.DataFrame()
train["pred"]=ss.inverse_transform(trainPredict.flatten())
train["real"]=ss.inverse_transform(trainX[:,:,0])
train["date"]=X_train.index[:-5]
train.iplot(kind="line", x="date")